# Modeling with recursive feature elemimation (rfe)
* usage of rfe to remove features and model
* usage of selekbest with stepwise approach to select best parameters
* Perform with 75-25 split (156, 52 observations)
* check adjusted r2 values to see whether significant difference between r2 and adjusted r2

### goals
* develop a model that effectively predicts 2019 AQI pollution levels
* fit model with training dataset
* try grid search (b/c we have a small dataset (maybe experiemnt with random grid))


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFECV, RFE
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn import preprocessing, svm
from sklearn.metrics import r2_score, accuracy_score

#Use to ignore convergence warnings
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.exceptions import ConvergenceWarning
from sklearn.exceptions import FitFailedWarning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=FitFailedWarning)


# pd.set_option('display.max_columns', None)
# pd.reset_option('max_rows')
# np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

In [2]:
# formula for adjusted R
def get_adjusted_r2(r2, X):
    """ Takes the r2 score and input variables and returns adjusted r2 score"""
    num_obs = len(X)
    num_predictors = X.shape[1]
    adj_r2 = 1 - (((1 - r2) * (num_obs - 1)) / (num_obs - num_predictors - 1))
    return adj_r2

In [3]:
# import X and y training and test sets

X = pd.read_csv('../../data/train_test/X_alt')

X_train = pd.read_csv('../../data/train_test/X_train_scaled_alt')

X_test = pd.read_csv('../../data/train_test/X_test_scaled_alt')

y = pd.read_csv('../../data/train_test/y_alt')

y_train = pd.read_csv('../../data/train_test/y_train_alt')

y_test = pd.read_csv('../../data/train_test/y_test_alt')


# Form a baseline for comparison


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
#

In [5]:
# # because we negative observation we cannot perform logarithmic scale and will instead perform standardization
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)

In [6]:
# scale test data using the scaler fitted from the training set

X_test_scaled = scaler.transform(X_test)

In [7]:
# Changed scaled X's from numpy array to dataframe and assign scaled data to X_train and X_test

# retain X_train_scaled as a dataframe
X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
display(X_train)

# retain X_test_scale as a dataframe
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)
display(X_test)


,AQI_2017_2018_diff,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018,HWAC_FEMALE_ratio_2018,HBAC_MALE_ratio_2018,HBAC_FEMALE_ratio_2018,HIAC_MALE_ratio_2018,HIAC_FEMALE_ratio_2018,HAAC_MALE_ratio_2018,HAAC_FEMALE_ratio_2018,HNAC_MALE_ratio_2018,HNAC_FEMALE_ratio_2018
29,2.705519,0.232787,0.160397,0.184132,-0.708151,-0.032929,-0.052244,-0.012659,-0.358545,-0.408037,...,-0.265623,-0.239267,-0.478360,-0.454617,-0.068953,-0.060159,0.727062,0.731266,0.338143,0.135153
19,-0.535302,-0.089040,0.214868,-0.808003,-1.896018,-0.366625,-0.255945,-0.478881,-0.358545,-0.272170,...,1.966411,1.747236,0.166351,0.127846,1.967471,1.962093,1.699092,1.665187,1.197938,1.248202
55,0.776477,-0.251222,-0.384063,0.364611,-0.114218,0.171585,0.192251,0.148936,0.144708,0.203365,...,-0.672893,-0.661165,-0.178878,-0.228808,-0.770089,-0.794749,-0.693480,-0.659929,-0.663461,-0.642367
93,0.021271,-0.363063,-0.509996,0.406645,0.182749,0.164637,0.192520,0.134550,0.144708,0.271299,...,0.612703,0.475619,0.255883,0.181816,0.435487,0.295619,-0.183886,-0.152873,0.157864,0.111195
181,0.105938,0.161016,0.018098,0.375286,0.776682,-0.053591,-0.097600,-0.007554,-0.358545,-0.408037,...,-0.779422,-0.761439,-0.542726,-0.507942,-0.881762,-0.876552,-0.721289,-0.695615,-0.577650,-0.600115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.497668,0.973573,0.912621,0.124417,0.479716,-0.753437,-0.800489,-0.699322,-0.861798,-0.883572,...,-0.422983,-0.395521,0.338929,0.261525,-0.683352,-0.676884,-0.550425,-0.545502,-0.561014,-0.537119
14,0.352561,-0.026907,0.191880,-0.583155,-0.411185,-1.187242,-1.037827,-1.333915,-0.789905,-0.679772,...,0.332722,0.343431,0.038756,0.040298,0.290977,0.296688,2.306325,2.301282,1.658731,1.537832
92,-1.339379,-0.376250,-0.538106,0.447011,0.479716,0.511314,0.493196,0.526518,1.726361,1.562038,...,-0.369061,-0.356314,-0.039043,0.065903,-0.094597,-0.083728,-0.452073,-0.398921,-0.590935,-0.571752
179,1.177203,-0.268594,-0.453151,0.503390,0.776682,0.239330,0.237250,0.239805,0.432282,0.407166,...,-0.801500,-0.772961,-0.597171,-0.555449,0.951338,0.820300,-0.587588,-0.652921,-0.652612,-0.691276


,AQI_2017_2018_diff,Civilian_labor_force_2017_2018_diff,Employed_2017_2018_diff,Unemployed_2017_2018_diff,Unemployment_rate_2017_2018_diff,"Poverty Estimate, All Ages_2017_2018_diff",90% CI LB All Ages_2017_2018_diff,90% CI UB All Ages_2017_2018_diff,"Poverty Percent, All Ages_2017_2018_diff",90% CI LB percent_2017_2018_diff,...,HWAC_MALE_ratio_2018,HWAC_FEMALE_ratio_2018,HBAC_MALE_ratio_2018,HBAC_FEMALE_ratio_2018,HIAC_MALE_ratio_2018,HIAC_FEMALE_ratio_2018,HAAC_MALE_ratio_2018,HAAC_FEMALE_ratio_2018,HNAC_MALE_ratio_2018,HNAC_FEMALE_ratio_2018
161,1.179983,-0.049097,0.089435,-0.367981,-1.005118,-0.012268,-0.026032,0.002099,-0.070972,-0.136303,...,-0.322966,-0.350746,-0.325545,-0.323909,0.387984,0.393480,-0.523710,-0.498360,-0.598638,-0.468140
15,-0.514207,-0.155612,-0.277120,0.330583,-0.708151,0.155220,0.167292,0.141604,0.000922,-0.000435,...,-0.321775,-0.304197,-0.540054,-0.544634,0.062488,-0.037553,0.107840,0.028584,-0.024577,-0.011896
73,-0.439342,0.098756,0.402143,-0.814008,-1.005118,-0.896515,-0.687143,-1.107440,-0.789905,-0.611838,...,-0.651537,-0.654134,-0.057482,-0.040492,-0.855003,-0.845808,-0.659433,-0.623406,-0.890726,-0.851071
96,0.772088,-0.380181,-0.551224,0.471697,-0.114218,0.145986,0.148416,0.142439,-0.214758,-0.272170,...,-0.882960,-0.855283,-0.588543,-0.537553,-0.887276,-0.937340,-0.744554,-0.698361,-0.847878,-0.951074
166,-0.232902,-0.437497,-0.518741,0.234172,-1.599052,0.165825,0.159956,0.170748,0.216602,0.135432,...,-0.932549,-0.898605,-0.589867,-0.553735,-1.025117,-1.003201,-0.874986,-0.837730,-0.975171,-0.965515
9,1.005701,0.734548,0.493220,0.615480,1.073649,0.342821,0.230988,0.456440,0.072815,-0.136303,...,0.678221,0.431181,-0.084002,-0.104289,2.280474,1.757984,0.414438,0.330372,0.385620,0.320694
100,-0.677657,-0.381323,-0.557096,0.484374,0.776682,0.045695,0.016730,0.075425,-0.718011,-0.883572,...,-0.932809,-0.897397,-0.664508,-0.631720,-1.015975,-0.996387,-0.825328,-0.820296,-1.069600,-0.937393
135,0.119467,-0.633535,-0.372569,-0.671893,0.182749,-1.402544,-1.288405,-1.511103,-0.358545,-0.272170,...,0.017514,0.116367,3.298175,3.480676,0.444916,0.627653,0.434853,0.596136,0.834306,1.091849
18,2.321711,-0.424943,-0.535482,0.311902,-0.411185,0.283487,0.207012,0.360838,0.647961,0.203365,...,3.640662,3.595297,-0.007260,0.009218,1.475927,1.546005,2.439127,2.590405,1.329744,1.435957
148,-0.946933,-0.373714,-0.513369,0.387630,-1.302085,0.174327,0.198245,0.148287,0.144708,0.271299,...,-0.899900,-0.916755,-0.496788,-0.618729,-0.911233,-1.017279,-0.864162,-0.871000,-1.063651,-0.959654


In [8]:
# Perform features selection with Recursive feature elemination cross validated

lasso = Lasso()
selector = RFECV(lasso, step=1, cv=5)
selector.fit(X_train, y_train)
print(selector.support_)
print(selector.ranking_)


[False False False ... False False False]
[1310 1309 1308 ... 1080 1082 1083]


In [9]:
rfe_col = X_train.columns[selector.support_]
rfe_col

Index(['HAA_MALE_ratio_2018_2019_diff', 'HWA_MALE_ratio_2018'], dtype='object')

In [10]:
X_train_rfe = X_train[rfe_col]
X_test_rfe = X_test[rfe_col]

## Develop baseline with dummy regressor
* run dummy regressor with strategy mean and median


In [11]:
# with mean
dummy = DummyRegressor(strategy='mean')

dummy.fit(X_train_rfe, y_train)

dummy.score(X_test_rfe, y_test)


-0.014729087673823082

In [12]:
# with median
dummy = DummyRegressor(strategy='median')

dummy.fit(X_train_rfe, y_train)

dummy.score(X_test_rfe, y_test)

-0.006734833218899672

### Run best features from RFE in a randomized grid


In [13]:
param_grid = {'alpha':np.arange(0.1, 1.1, step=0.00000001),
              'selection':['random', 'cyclic'],
              'positive':[True, False],
              'fit_intercept':[True,False], 'random_state':[42],
              'normalize':[True,False], 'warm_start':[True,False]}
lasso = Lasso()
lasso_grid = RandomizedSearchCV(lasso, param_grid, cv=5, random_state=42)
lasso_grid.fit(X_train_rfe, y_train)
print('best score: ', lasso_grid.best_score_)
print('adjusted r2 score:', get_adjusted_r2(float(lasso_grid.best_score_), X_train_rfe))
print('best estimator', lasso_grid.best_estimator_)
tmp = lasso_grid.best_score_

best score:  0.1994193003432788
adjusted r2 score: 0.1889541931582236
best estimator Lasso(alpha=0.1779584999589661, positive=True, random_state=42, warm_start=True)


In [14]:
lasso_best = lasso_grid.best_estimator_
lasso_best.fit(X_train_rfe, y_train)

# this retrieves correlation of determination value
print('R^2', lasso_best.score(X_test_rfe, y_test))
print('Adjusted R2;', get_adjusted_r2(lasso_best.score(X_test_rfe, y_test), X_test_rfe))

R^2 0.3896632598382048
Adjusted R2; 0.3647515561581316


### Evaluation
* Although scoring is weak, the differnce between r2 and adjusted r2 are similar and not significantly different in both test and train set
* It appears that RFE onyl uses 2 featueres

### Run modified select K best for feature selection
* run select_k_best for every features in df
* compute hyperaparamter tuning with every number of columns using select_K_best
* store and r2 and adjusted r2 scores
* evaluate which number of features yielded best r2 and adjusted r2 scores

In [15]:
lasso_r2_scores = []
lasso_adj_r2_scores = []
lasso_r2_test_scores = []
lasso_adj_r2_test_scores = []

for num in range(1,101):
    select_k_best = SelectKBest(f_regression, k=num)
    X_train_k_best = select_k_best.fit_transform(X_train, y_train)

    best_k_cols = X_train.columns[select_k_best.get_support()]
    best_k_cols

    param_grid = {'alpha':np.arange(0.1, 1.1, step=0.00000001),
              'selection':['random', 'cyclic'],
              'positive':[True, False],
              'fit_intercept':[True,False], 'random_state':[42],
              'normalize':[True,False], 'warm_start':[True,False]}
    lasso = Lasso()
    lasso_grid = RandomizedSearchCV(lasso, param_grid, cv=5, random_state=42)
    lasso_grid.fit(X_train_k_best, y_train)
    # append best scores at bottom
    lasso_r2_scores.append(lasso_grid.best_score_)
    lasso_adj_r2_scores.append(get_adjusted_r2(float(lasso_grid.best_score_), X_train_k_best))

    lasso_best = lasso_grid.best_estimator_
    lasso_best.fit(X_train_k_best, y_train)

In [16]:
max_r2_score = max(lasso_r2_scores)
max_r2_score_index = lasso_r2_scores.index(max_r2_score) + 1
# prints that at 60 features we get the best r2 score
print(max_r2_score, " best r2 score at number features", max_r2_score_index)

0.24186217393142692  best r2 score at number features 63


In [17]:
# for adjusted score it also appears to be 60
max_adj_r2_score_index = lasso_adj_r2_scores.index(max(lasso_adj_r2_scores)) + 1
print(max(lasso_adj_r2_scores), " best adjusted r2 score at number features", max_adj_r2_score_index)

0.15735459747553326  best adjusted r2 score at number features 1


## Run grid with 61 predictor variables, the number that had best r2 score

In [24]:
select_k_best = SelectKBest(f_regression, k=max_r2_score_index)
X_train_k_best = select_k_best.fit_transform(X_train, y_train)

best_k_cols = X_train.columns[select_k_best.get_support()]
best_k_cols
param_grid = {'alpha':np.arange(0.1, 1.1, step=0.00000001),
          'selection':['random', 'cyclic'],
          'positive':[True, False],
          'fit_intercept':[True,False], 'random_state':[42],
          'normalize':[True,False], 'warm_start':[True,False]}
lasso = Lasso()
lasso_grid = RandomizedSearchCV(lasso, param_grid, cv=5, random_state=42)
lasso_grid.fit(X_train_k_best, y_train)
# append best scores at bottom
print(lasso_grid.best_score_)
print(get_adjusted_r2((lasso_grid.best_score_), X_train_k_best))

0.24186217393142692
-0.2772974243546611


In [25]:
lasso_best = lasso_grid.best_estimator_
lasso_best.fit(X_train_k_best, y_train)
# this retrieves correlation of determination value
print('R^2', lasso_best.score(X_test[best_k_cols], y_test))
print('Adjusted R2;', get_adjusted_r2(lasso_best.score(X_test[best_k_cols], y_test), X_test[best_k_cols]))

R^2 0.39734778794619097
Adjusted R2; 3.561271901228688


### Evaluation
* Running with 63 features has around .4 score
* adjusted R2 has a value over 1; this is because running on a test set of 52 observations and 63 predictor variables is an issue
where we have more predictors than observations

## Run grid with 1 predictor variables, the number that had best adjusted r2 score

In [21]:

# evaluation for 20 features
select_k_best = SelectKBest(f_regression, k=max_adj_r2_score_index)
X_train_k_best = select_k_best.fit_transform(X_train, y_train)

best_k_cols = X_train.columns[select_k_best.get_support()]
best_k_cols
param_grid = {'alpha':np.arange(0.1, 1.1, step=0.00000001),
          'selection':['random', 'cyclic'],
          'positive':[True, False],
          'fit_intercept':[True,False], 'random_state':[42],
          'normalize':[True,False], 'warm_start':[True,False]}
lasso = Lasso()
lasso_grid = RandomizedSearchCV(lasso, param_grid, cv=5, random_state=42)
lasso_grid.fit(X_train_k_best, y_train)
# append best scores at bottom
print(lasso_grid.best_score_)
print(get_adjusted_r2((lasso_grid.best_score_), X_train_k_best))

0.162791019427304
0.15735459747553326


In [22]:
# Evaluation for 61 features

lasso_best = lasso_grid.best_estimator_
lasso_best.fit(X_train_k_best, y_train)

# this retrieves correlation of determination value
print('R^2', lasso_best.score(X_test[best_k_cols], y_test))
print('Adjusted R2;', get_adjusted_r2(lasso_best.score(X_test[best_k_cols], y_test), X_test[best_k_cols]))


R^2 0.4066554892685488
Adjusted R2; 0.39478859905391983


### Evaluation
* Running with 1 features appears to yeild the best r2 score and adjusted r2 score for a 75 / 30 split
